In [6]:
import os
import re
import time
import imageio
import json
import glob
from pathlib import Path
from datetime import datetime, timedelta

import geojsoncontour
import numpy as np
import pandas as pd

import wrf
from netCDF4 import Dataset as NetCDFFile

import folium
from folium import plugins
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import branca.colormap as cm
from matplotlib import colors as mcolors

from shapely.geometry import shape, Point

In [7]:
nc_file = NetCDFFile('../wrf_output')
time_size = nc_file.dimensions['Time'].size

In [8]:
with open('../Limite Distrito de Barranquilla.geojson') as f:
    baq_geojson = json.load(f)
    baq_polygon = shape(baq_geojson['features'][0]['geometry'])

In [9]:
# https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.wrf.getvar.html
height = wrf.getvar(nc_file, 'height', timeidx=1)
Var_all = wrf.getvar(nc_file, 'tc', timeidx=1)

var = wrf.interplevel(Var_all, height, 150)

(lats, lons) = wrf.latlon_coords(var)

var

<xarray.DataArray 'temp_interp' (south_north: 81, west_east: 90)>
array([[      nan,       nan,       nan, ..., 24.9206  , 24.932354,
        24.944311],
       [      nan,       nan,       nan, ..., 24.916996, 24.930206,
        24.941664],
       [      nan,       nan,       nan, ..., 24.914343, 24.927904,
        24.9391  ],
       ...,
       [24.678436, 24.6746  , 24.671469, ..., 24.711538, 24.717651,
        24.722378],
       [24.677052, 24.676365, 24.672924, ..., 24.709002, 24.715117,
        24.719128],
       [24.675585, 24.67364 , 24.671665, ..., 24.704475, 24.710152,
        24.715961]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -74.96 -74.96 ... -74.68 -74.68
    XLAT     (south_north, west_east) float32 10.85 10.85 10.85 ... 11.11 11.11
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-04-05T03:00:00
    level    int64 150
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:          degC
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LatLon(stand_lon=-74.8219985961914, moad_cen_lat=10.98099...
    _FillValue:     9.969209968386869e+36
    missing_value:  9.969209968386869e+36
    vert_units:     m

In [10]:
height

<xarray.DataArray 'height' (bottom_top: 34, south_north: 81, west_east: 90)>
array([[[  178.53589 ,   176.79404 ,   185.58444 , ...,    28.497963,
            28.615925,    28.745777],
        [  174.03557 ,   169.25801 ,   172.39536 , ...,    28.423862,
            28.412474,    28.425785],
        [  174.42424 ,   165.79105 ,   164.2004  , ...,    28.419884,
            28.414751,    28.41988 ],
        ...,
        [   26.449923,    26.449472,    26.449406, ...,    26.43198 ,
            26.433743,    26.435192],
        [   26.450056,    26.44987 ,    26.449575, ...,    26.43276 ,
            26.433895,    26.435421],
        [   26.450182,    26.449898,    26.449615, ...,    26.435663,
            26.435377,    26.435644]],

       [[  238.3011  ,   236.56282 ,   245.33049 , ...,    88.66125 ,
            88.78066 ,    88.911964],
        [  233.81358 ,   229.04597 ,   232.17242 , ...,    88.58825 ,
            88.577484,    88.592674],
        [  234.20158 ,   225.5886  ,   223.99707 , ...,    88.58498 ,
            88.58038 ,    88.58692 ],
...
        [19327.71    , 19327.658   , 19327.564   , ..., 19326.494   ,
         19325.846   , 19324.629   ],
        [19327.53    , 19327.48    , 19327.408   , ..., 19325.76    ,
         19325.29    , 19324.568   ],
        [19327.342   , 19327.365   , 19327.38    , ..., 19324.658   ,
         19324.578   , 19324.5     ]],

       [[20175.945   , 20175.957   , 20175.78    , ..., 20175.262   ,
         20175.209   , 20175.166   ],
        [20176.121   , 20176.01    , 20175.904   , ..., 20175.295   ,
         20175.25    , 20175.17    ],
        [20176.205   , 20176.06    , 20175.977   , ..., 20175.223   ,
         20175.213   , 20175.168   ],
        ...,
        [20175.785   , 20175.77    , 20175.756   , ..., 20173.512   ,
         20172.988   , 20171.973   ],
        [20175.596   , 20175.623   , 20175.666   , ..., 20172.887   ,
         20172.482   , 20171.898   ],
        [20175.404   , 20175.416   , 20175.42    , ..., 20171.979   ,
         20171.895   , 20171.81    ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -74.96 -74.96 ... -74.68 -74.68
    XLAT     (south_north, west_east) float32 10.85 10.85 10.85 ... 11.11 11.11
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-04-05T03:00:00
Dimensions without coordinates: bottom_top, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  model height - [MSL] (mass grid)
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LatLon(stand_lon=-74.8219985961914, moad_cen_lat=10.9809999...

In [11]:
var_geo_bounds = wrf.geo_bounds(var)

In [12]:
len(np.arange(var_geo_bounds.bottom_left.lat, var_geo_bounds.top_right.lat, 0.01))

26

In [13]:
len(np.arange(var_geo_bounds.bottom_left.lon, var_geo_bounds.top_right.lon, 0.02))

15

In [14]:
np.arange(var_geo_bounds.bottom_left.lat, var_geo_bounds.top_right.lat, 0.01)

array([10.85280037, 10.86280037, 10.87280037, 10.88280037, 10.89280037,
       10.90280037, 10.91280037, 10.92280037, 10.93280037, 10.94280037,
       10.95280037, 10.96280037, 10.97280037, 10.98280037, 10.99280037,
       11.00280037, 11.01280037, 11.02280037, 11.03280037, 11.04280037,
       11.05280037, 11.06280037, 11.07280037, 11.08280037, 11.09280037,
       11.10280037])

In [15]:
np.arange(var_geo_bounds.bottom_left.lon, var_geo_bounds.top_right.lon, 0.02)

array([-74.96463013, -74.94463013, -74.92463013, -74.90463013,
       -74.88463013, -74.86463013, -74.84463013, -74.82463013,
       -74.80463013, -74.78463013, -74.76463013, -74.74463013,
       -74.72463013, -74.70463013, -74.68463013])

In [28]:
def get_data(nc_file: NetCDFFile, timeidx: int):
    desiredlev = 150
    height = wrf.getvar(nc_file, 'height', timeidx=timeidx)

    u_all = wrf.getvar(nc_file, 'ua', timeidx=timeidx)
    v_all = wrf.getvar(nc_file, 'va', timeidx=timeidx)
    T_all = wrf.getvar(nc_file, 'tc', timeidx=timeidx)
    P_all = wrf.getvar(nc_file, 'pressure', timeidx=timeidx)
    pw = wrf.getvar(nc_file, 'pw', timeidx=timeidx)

    P = wrf.interplevel(P_all, height, desiredlev)
    T = wrf.interplevel(T_all, height, desiredlev)
    u = wrf.interplevel(u_all, height, desiredlev)
    v = wrf.interplevel(v_all, height, desiredlev)

    data = {
        'pwater': ('Precipitable Water (kg/m2)', pw),
        'temp': ('Temperature (C)', T),
        'wind': ('Wind speed (m/s)', np.sqrt(u ** 2 + v ** 2)),
        'uwind': ('U wind speed (m/s)', u),
        'vwind': ('V wind speed (m/s)', v),
        'press': ('Pressure (hPa)', P)
    }

    return data


def geojson_title_to_float(title):
    result = re.search(
        r"([-]?([0-9]*[.])?[0-9]+)-([-]?([0-9]*[.])?[0-9]+)", title)
    groups = result.groups()

    value = np.median([float(groups[0]), float(groups[2])])

    return value


def gj_to_df(gj):
    gj_data = np.zeros([len(gj['features']), 2])

    for i in range(len(gj['features'])):
        gj['features'][i]['id'] = i
        gj_data[i, 0] = i
        gj_data[i, 1] = geojson_title_to_float(
            gj['features'][i]['properties']['title'])

    df = pd.DataFrame(gj_data, columns=['id', 'variable'])

    return df


def build_gif_frame(lats, lons, caption, variable, date):
    contour = plt.contourf(lons, lats, variable, cmap=plt.cm.jet)

    gj = json.loads(geojsoncontour.contourf_to_geojson(
        contourf=contour, ndigits=4, unit='m'))
    df_contour = gj_to_df(gj)
    
    zmin = df_contour.variable.min() - df_contour.variable.median() / 10
    zmax = df_contour.variable.max() + df_contour.variable.median() / 10

    trace = go.Choroplethmapbox(
        geojson=gj,
        locations=df_contour.id,
        z=df_contour.variable,
        zmin=zmin,
        zmax=zmax,
        colorscale='jet',
        marker_line_width=0.1,
        marker=dict(opacity=0.2)
    )

    layout = go.Layout(
        title=f"{caption} - {date} GMT-5",
        title_x=0.5,
        width=600,
        margin=dict(t=26, b=0, l=0, r=0),
        font=dict(color='black', size=10),
        mapbox=dict(
            center=dict(
                lat=lats.mean().item(0),
                lon=lons.mean().item(0)
            ),
            zoom=11,
            style='carto-positron'
        )
    )

    fig = go.Figure(data=[trace], layout=layout)

    return fig


def get_image(timeidx: int, nc_var: str, start_date: datetime, created_at: datetime):
    date = start_date + timedelta(hours=timeidx * 3) - timedelta(hours=5)

    data = get_data(nc_file, timeidx)

    (caption, variable) = data[nc_var]
    (lats, lons) = wrf.latlon_coords(variable)

    fig = build_gif_frame(lats, lons, caption, variable, date)

    png_file = f"{nc_var}_{timeidx}.png"
    try:
        fig.write_image(png_file)
    except Exception:
        return None

    img = imageio.imread(png_file)
    os.remove(png_file)

    if timeidx == time_size - 1:
        build_folium_map(lats, lons, caption, variable, date, created_at)

    return img


def build_folium_map(lats, lons, caption, variable, start_date, created_at):
    vmin = variable.min() - variable.median() / 10
    vmax = variable.max() + variable.median() / 10

    contour = plt.contourf(lons, lats, variable, cmap=plt.cm.jet, vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(contour)

    gj = json.loads(geojsoncontour.contourf_to_geojson(
        contourf=contour, ndigits=4, unit='m'))

    f_map = folium.Map(
        location=[lats.mean(), lons.mean()],
        tiles='Cartodb Positron',
        zoom_start=12
    )

    folium.GeoJson(
        gj,
        style_function=lambda x: {
            'color': x['properties']['stroke'],
            'weight': x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity': 0.3
        },
        name='contour'
    ).add_to(f_map)
    
    colormap = cm.LinearColormap(
        colors=['darkblue', 'blue', 'cyan', 'green', 'greenyellow', 'yellow', 'orange', 'red', 'darkred'],
        index=np.array(cbar.values),
        vmin=cbar.values[0],
        vmax=cbar.values[len(cbar.values) - 1],
        caption=caption
    )
    f_map.add_child(colormap)
    
    folium.GeoJson(
        baq_geojson,
        style_function=lambda x: {
            'color': 'rgb(12, 131, 242)',
            'fillColor': 'rgba(255, 0, 0, 0)'
        },
        name='baq_map'
    ).add_to(f_map)
    
    var_geo_bounds = wrf.geo_bounds(variable)
    for lat in np.arange(var_geo_bounds.bottom_left.lat, var_geo_bounds.top_right.lat, 0.01):
        for lon in np.arange(var_geo_bounds.bottom_left.lon, var_geo_bounds.top_right.lon, 0.02):
            if baq_polygon.contains(Point(lon, lat)):
                x, y = wrf.ll_to_xy(nc_file, lat, lon)
                value = round(variable[x.item(0), y.item(0)].values.item(0), 2)
                folium.Marker(
                    location=[lat, lon],
                    popup=None,
                    icon=folium.DivIcon(
                    html=f"""<span style="font-size: 16px; color: yellow; -webkit-text-stroke: 1px black;">{value}</span>""")
                ).add_to(f_map)
        
    f_map.get_root().html.add_child(folium.Element('<p style="text-align:center;font-size:14px;margin:4px">{} GMT-5</p>'.format(start_date)))

    f_map.save(f"{nc_var}_{created_at}.html")

In [29]:
nc_var = 'temp'
created_at = datetime.strptime("2022-03-26 15:00", '%Y-%m-%d %H:%M')
start_date = datetime.strptime("2022-03-26 18", '%Y-%m-%d %H')

results = [get_image(timeidx, nc_var, start_date, created_at) for timeidx in range(time_size)]

imageio.mimwrite(f"{nc_var}_{created_at}.gif", [img for img in results if img is not None], fps=0.5)